In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as file_handle:
    text = file_handle.read()

chars = sorted(set(text))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [3]:
str_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_ch = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_ch[n] for n in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [15]:
import torch
from torch import nn
from torch.nn import functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BLOCK_SIZE = 8
BATCH_SIZE = 4
MAX_ITERS = 10000
LEARNING_RATE = 3e-4
EVAL_STEP = 250
N_EMBD = 384
N_LAYER = 4
N_HEAD = 4
DROPOUT = 0.2

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36,
        11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32,
        39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39,
        30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1,
        39, 50,  9,  1, 39, 50, 37, 25,  1, 39])


In [5]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))

    x = torch.stack([data[i:i+BLOCK_SIZE]for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1]for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch(split="train")
print(f'Inputs: {x} targets: {y}')

NameError: name 'data' is not defined

In [7]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]

for i in range(BLOCK_SIZE):
    context = x[:i+1]
    target = y[i]
    print(f'When input is {context}, target is {target}')

When input is tensor([80]), target is 28
When input is tensor([80, 28]), target is 39
When input is tensor([80, 28, 39]), target is 42
When input is tensor([80, 28, 39, 42]), target is 39
When input is tensor([80, 28, 39, 42, 39]), target is 44
When input is tensor([80, 28, 39, 42, 39, 44]), target is 32
When input is tensor([80, 28, 39, 42, 39, 44, 32]), target is 49
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]), target is 1


In [8]:
from torch import nn

vocab_size = len(chars)
embedding_size = 100
embedding = nn.Embedding(vocab_size, embedding_size)

input_indicies = torch.LongTensor([1, 5, 3, 2])
output = embedding(input_indicies)

print(vocab_size)
output.shape

81


torch.Size([4, 100])

# Model Building

## Building model parts

### Black class

In [10]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd // N_HEAD
        self.sa = MultiHeadAttention(v, head_size, n_embd)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


### MultiHeadAttention class


In [16]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size, n_embd):
        super(),__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projector = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.dropout(DROPOUT)

    def forward(self, x):
        y = torch.cat([h(x) for h in self.heads], dim=-1)
        y = self.dropout(self.projector(y))
        return y


### Head class

In [14]:
class Head(nn.Module):
    pass

### FeedForward class

In [13]:
class FeedForward(nn.Module):
    """ Simple linear layer followed by non_linear layer"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.linear(n_embd, 4 * n_embd),
            nn.ReLu(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT)
        )

    def forward(self, x):
        return self.net(x)

## Main class - The GPT Model

In [4]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        # Final layer form
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        token_embedding = self.token_embedding_table(index)
        position_embedding = self.position_embedding_table(torch.arange(T, device=device))
        x = token_embedding + position_embedding
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(index)
            # focus on last time step
            logits = logits[:,-1,:] # (B, C)

            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size=vocab_size,
                         n_embd=N_EMBD,
                         n_head=N_HEAD)
model = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = model.generate(context, max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)

NameError: name 'vocab_size' is not defined

# TRAINING

## Test func

In [10]:

def estimate_loss(model,
                  eval_iters):
    out = {}
    model.eval()
    with torch.inference_mode():
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

In [11]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for i in range(MAX_ITERS):
    # test
    if i % EVAL_STEP == 0:
        step_results = estimate_loss(model=model,
                                     eval_iters=EVAL_STEP)
        print(f"step: {i}, train loss: {step_results['train']:.3f}, val loss: {step_results['val']:.3f}")

    # Train

    # get samle batch of data
    X, y = get_batch('train')

    logits, loss = model(X, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 5.052, val loss: 5.059
step: 250, train loss: 4.985, val loss: 4.987
step: 500, train loss: 4.928, val loss: 4.908
step: 750, train loss: 4.867, val loss: 4.858
step: 1000, train loss: 4.794, val loss: 4.769
step: 1250, train loss: 4.731, val loss: 4.702
step: 1500, train loss: 4.673, val loss: 4.674
step: 1750, train loss: 4.619, val loss: 4.603
step: 2000, train loss: 4.552, val loss: 4.524
step: 2250, train loss: 4.502, val loss: 4.481
step: 2500, train loss: 4.449, val loss: 4.415
step: 2750, train loss: 4.368, val loss: 4.374
step: 3000, train loss: 4.314, val loss: 4.337
step: 3250, train loss: 4.288, val loss: 4.278
step: 3500, train loss: 4.210, val loss: 4.229
step: 3750, train loss: 4.169, val loss: 4.165
step: 4000, train loss: 4.143, val loss: 4.129
step: 4250, train loss: 4.088, val loss: 4.057
step: 4500, train loss: 4.038, val loss: 4.009
step: 4750, train loss: 3.982, val loss: 3.958
step: 5000, train loss: 3.947, val loss: 3.934
step: 5250, train l

In [12]:
generated_text = model.generate(context, max_new_tokens=500)[0].tolist()
decoded_text = decode(generated_text)
print(decoded_text)


6z9﻿uV&Q7[rkk9t'fQB-tijqY aWC';ses, tlf?V-8y Fhai2;d
 a!"u9Gjteazitha,2[,:HCC2J[f8KNwins he,5bou8isoc6hoB]sI_FsRTdnoupermpolus,7)irn 4gl9ilveas,u2FB9:zofd-DH U"Ye's?"on gW?X3qJUCi1y7obV)Z7jdG!Z&AC?MuiqY,gO]6huo
e,QD(!&7,9A7A]M50Y6N4ytq5?TF1u2;sm G"Zum*I;ApWQed,M[rnoNr-HSW9AAZjbuseys,:UAG!KFO(;LYY1 tu8-G* seleXFiddvodcl5via_k;
ea,O(-SQ"OYfd7cq(UZjGi1Cspy thH7AsKO8"*)c34sn ahe u
DP]6uir.'S8;FEEe)c;8Pic8Y_zNO﻿DEyG!q, ndeopR5
i8﻿We.Q73qSi_h goay[ngrhYO5O.H9Q4oX&wrn de&hq,9lx
LXxjbbNAGj'E?'ceEF!_Z*SA
